In [3]:
import keras, datetime
from keras.layers import Input, Dense
from keras.models import Model
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from keras.applications import mobilenet_v2
import numpy as np
import os

from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')


base_path = '/content/gdrive/My Drive/cat_classification/dataset'


# input image size & output size
img_size = 224
output_size = 2

start_time = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')


# load dataset
data_00 = np.load(base_path + '/CAT_00.npy', allow_pickle=True)
data_01 = np.load(base_path + '/CAT_01.npy', allow_pickle=True)
data_02 = np.load(base_path + '/CAT_02.npy', allow_pickle=True)
data_03 = np.load(base_path + '/CAT_03.npy', allow_pickle=True)
data_04 = np.load(base_path + '/CAT_04.npy', allow_pickle=True)
data_05 = np.load(base_path + '/CAT_05.npy', allow_pickle=True) # validationset

print('dataset load 완료')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
dataset load 완료


In [4]:

# train set
x_train = np.concatenate((data_00.item().get('imgs'), data_01.item().get('imgs'), data_02.item().get('imgs'), data_03.item().get('imgs'), data_04.item().get('imgs')), axis=0)
y_train = np.concatenate((data_00.item().get('lbs'), data_01.item().get('lbs'), data_02.item().get('lbs'), data_03.item().get('lbs'), data_04.item().get('lbs')), axis=0)


# test set
x_test = np.concatenate((data_05.item().get('imgs')), axis=0)
y_test = data_05.item().get('lbs')


# 정규화
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.


# reshape
x_train = np.reshape(x_train, (-1, img_size, img_size, 3))
x_test = np.reshape(x_test, (-1, img_size, img_size, 3))
y_train = np.reshape(y_train, (-1, 1))
y_test = np.reshape(y_test, (-1, 1))


print('정규화 완료')

정규화 완료


In [5]:
inputs = Input(shape=(img_size, img_size, 3))

mobilenetv2_model = mobilenet_v2.MobileNetV2(input_shape=(img_size, img_size, 3), alpha=1.0, include_top=True, weights=None, input_tensor=inputs, classes=2)

net = Dense(128, activation='relu')(mobilenetv2_model.layers[-1].output)
net = Dense(64, activation='relu')(net)
net = Dense(output_size, activation='sigmoid')(net)

model = Model(inputs=inputs, outputs=net)

model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
_______________________________________________________________________________________

In [ ]:
# training
model.compile(optimizer=keras.optimizers.Adam(), loss='mse')

model.fit(x_train, y_train, 
          epochs=50, 
          batch_size=32, 
          shuffle=True,
          validation_data=(x_test, y_test), 
          verbose=1,
  callbacks=[
    TensorBoard(log_dir='logs/%s' % (start_time)),
    ModelCheckpoint('./models/%s.h5' % (start_time), monitor='val_loss', verbose=1, save_best_only=True, mode='auto'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, mode='auto')
  ]
)

Epoch 1/50
